<a href="https://colab.research.google.com/github/maksbd04/RK1/blob/master/%D0%91%D0%BE%D1%82_%D0%BA%D0%B0%D1%84%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyTelegramBotAPI
!pip install peewee


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from peewee import *

db = SqliteDatabase('my.db')

class Cafe(Model):
    name = CharField()
    address = CharField()
    image = CharField()
    rating = IntegerField()

    class Meta:
        database = db

class Food(Model):
    name = CharField()
    image = CharField()
    price = IntegerField()

    class Meta:
        database = db


class Menu(Model):
    food = ForeignKeyField(Food)
    cafe = ForeignKeyField(Cafe)

    class Meta:
        database = db

db.connect()

db.drop_tables([Cafe,Food,Menu])
db.create_tables([Cafe,Food,Menu])

coffe = Cafe(name='Кофейня', address='Baumanskaya', image="coffe.png", rating=1)
vkus = Cafe(name='Макдак', address='Tverskaya', image="vkus.png",rating=3)
kfc = Cafe(name='КФЦ', address='Пушкина', image="kfc.png",rating=3)


coffe.save()
vkus.save()
kfc.save()


americano = Food(name='Американо', image='americano.png', price=90)
cheeseburger = Food(name='Чизбургер', image='cheeseburger.png', price=120)
cookies = Food(name='Печенье', image='cookies.png', price=80)

americano.save()
cheeseburger.save()
cookies.save()

coffe_americano = Menu(food=americano,cafe=coffe)
coffe_cookies = Menu(food=cookies,cafe=coffe)
vkus_cheeseburger = Menu(food=cheeseburger,cafe=vkus)
vkus_americano = Menu(food=americano,cafe=vkus)

coffe_americano.save()
coffe_cookies.save()
vkus_cheeseburger.save()
vkus_americano.save()

1

In [ ]:
import telebot
from telebot import types

state = {"testuser":"cafe"}
state_cafe = {"testuser":"vkus"}
bot = telebot.TeleBot("6011328634:AAHF13KR8yhatc7BmW7S-LBdA8ZmxphDib8")

@bot.message_handler(commands=['start'])
def start_message(message):
    global state
    global state_cafe
    state[message.chat.username] ="cafe"
    state_cafe[message.chat.username] = "vkus"
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
    for c in Cafe.select():
      item_cafe=types.KeyboardButton(c.name)
      markup.add(item_cafe)
    audio = open('car.mp3', 'rb')
    bot.send_audio(message.from_user.id, audio)
    audio.close()
    bot.send_message(message.chat.id,"Привет, я помогу выбрать еду",reply_markup=markup)

@bot.message_handler(commands=['button'])
def button_message(message):
    global state
    global state_cafe
    state[message.chat.username] ="cafe"
    state_cafe[message.chat.username] = "vkus"
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
    for c in Cafe.select():
      item_cafe=types.KeyboardButton(c.name)
      markup.add(item_cafe)
    audio = open('car.mp3', 'rb')
    bot.send_audio(message.from_user.id, audio)
    audio.close()
    bot.send_message(message.chat.id,"Привет, я помогу выбрать еду",reply_markup=markup)

@bot.message_handler(content_types='text')
def message_reply(message):
    global state
    global state_cafe
    print("======DEBUG======")
    print(state)
    print("======DEBUG======")
    if state[message.chat.username] == "cafe":
      markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
      for c in Cafe.select():
        if c.name == message.text:
          ph = open(c.image,"rb")
          bot.send_photo(message.chat.id,ph)
          mystr = "Название: " + c.name + " Рейтинг: " + str(c.rating) + " Адрес:" + c.address
          bot.send_message(message.chat.id,mystr)
          state[message.chat.username] = "menu"
          state_cafe[message.chat.username] = c.name
          markup = types.InlineKeyboardMarkup()
          markup.row_width = 2
          for m in Menu.select():
            if m.cafe.name == state_cafe[message.chat.username]:
              print("!!!!!!!!!!!")
              food_str = m.food.name + ": " + str(m.food.price)
              markup.add(types.InlineKeyboardButton(food_str,callback_data=m.food.name))
          bot.send_message(message.chat.id,"Выберите блюдо",reply_markup=markup)

    if state[message.chat.username] == "menu":
        markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
        for m in Menu.select():
          if m.cafe.name == state_cafe:
            ph = open(m.food.image,"rb")
            bot.send_photo(message.chat.id,ph)
            mystr = "Ваш заказ: " + m.food.name + " Цена: " + str(m.food.price)
            bot.send_message(message.chat.id,mystr,reply_markup=markup)

        state[message.chat.username] = "cafe"
        for c in Cafe.select():
          item_cafe=types.KeyboardButton(c.name)
          markup.add(item_cafe)
        bot.send_message(message.chat.id,"Привет, я помогу выбрать еду",reply_markup=markup)




bot.infinity_polling()